In [1]:
import yaml
import requests

## Getting the data

Simply loading the data from the repo and load the yaml in memory. There are two version of the landscape. One light version with the crunchbase link and one with all the data extracted.

In [2]:
landscape_raw = requests.get("https://raw.githubusercontent.com/cncf/landscape/master/processed_landscape.yml")

In [3]:
landscape = yaml.safe_load(landscape_raw.content)['landscape']

## Parsing the categories

In [4]:
categories = { c['name']: [sub['name'] 
    for sub in c ['subcategories']] 
for c in landscape }

categories

{'Provisioning': ['Automation & Configuration',
  'Container Registry',
  'Security & Compliance',
  'Key Management'],
 'Runtime': ['Cloud Native Storage',
  'Container Runtime',
  'Cloud Native Network'],
 'Orchestration & Management': ['Scheduling & Orchestration',
  'Coordination & Service Discovery',
  'Remote Procedure Call',
  'Service Proxy',
  'API Gateway',
  'Service Mesh'],
 'App Definition and Development': ['Database',
  'Streaming & Messaging',
  'Application Definition & Image Build',
  'Continuous Integration & Delivery'],
 'Platform': ['Certified Kubernetes - Distribution',
  'Certified Kubernetes - Hosted',
  'Certified Kubernetes - Installer',
  'PaaS/Container Service'],
 'Serverless': ['Security',
  'Tools',
  'Framework',
  'Hosted Platform',
  'Installable Platform'],
 'Observability and Analysis': ['Monitoring',
  'Logging',
  'Tracing',
  'Chaos Engineering',
  'Continuous Optimization'],
 'Special': ['Kubernetes Certified Service Provider',
  'Kubernetes Trai

## A-to-Z split of the landscape

In [5]:
def get_only_letter(x: str, landscape: list):
    return { c['name']: { sub['name']: [item
            for item in sub['items'] if item['name'].startswith(x)]
        for sub in c ['subcategories']}
    for c in landscape }

In [6]:
for letter in range(ord('A'), ord('Z')+1):
    index = letter - ord('A')
    patial = get_only_letter(chr(letter), landscape)

    with open(f'../data/week_{str(index).zfill(2)}_{chr(letter)}.yaml', 'w+') as file:
        documents = yaml.dump(patial, file)
